<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Supabase - Update available models

**Tags**: #gsheet #openai #finetuning #ai #llm

**Author:** [Maxime Jublou](https://www.linkedin.com/in/maximejublou)

**Last update:** 2023-11-08 (Created: 2023-11-08)

**Description:** This notebook fine tuned a dataset from a Google spreadsheet with an OpenAI model to create your own model.

**References:**
- [OpenAI Fine-tuning](https://platform.openai.com/docs/guides/fine-tuning)

## Input

### Import libraries

In [ ]:
import naas
import pandas as pd
import psycopg2
import os
import naas_data_product

### Setup variables

In [ ]:
avatar_name = "Jeremy Ravenel"
output_dir = "."
print("Output directory:", output_dir)

avatar_model_alias = "naas/jeremy_linkedin_posts"
avatar_image = "https://cl-151.imagineapi.dev/assets/65c5b0bb-1507-454b-8da9-c1518478c683.png"
model = None

## Model

### Get model from output dir and create plugin output path

In [ ]:
if not model:
    files = os.listdir(output_dir)
    sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(output_dir, x)), reverse=True)
    for file in sorted_files:
        if file.startswith("ft:"):
            model = file
            print("Model:", model)
            break

### Connect to Supabase DB (Postgres)

In [ ]:
POSTGRES_HOST = "db.qsrivaytpfnitatvgfky.supabase.co"
POSTGRES_PORT = "5432"
POSTGRES_DATABASE = "postgres"
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = naas.secret.get('WORKSPACE_POSTGRES_PASSWORD')

class Postgres:
    
    def __init__(self, host, port, database, user, password):
        self.conn = psycopg2.connect(
            host=host,
            port=port,
            database=database,
            user=user,
            password=password
        )
        
        
    def __convert_pg_rows_to_dataframe(self, cursor):
        if not cursor.description:
            return None
        
        # Get the column names from the result set
        columns = [desc[0] for desc in cursor.description]

        # Get the data from the result set
        data = cursor.fetchall()

        # Create a DataFrame from the data and column names
        df = pd.DataFrame(data, columns=columns)

        return df.reset_index(drop=True)
    
    def query(self, q, commit=False):
        # Create a cursor object
        cur = self.conn.cursor()

        # Execute an SQL query
        cur.execute(q)

                
        if commit:
            self.conn.commit()
        
        rows = []
        if cur:
            # Fetch all rows from the result set
            rows = self.__convert_pg_rows_to_dataframe(cur)

        # Close the cursor and the connection
        cur.close()
        return rows
        
pg = Postgres(
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
    database=POSTGRES_DATABASE,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD
)

## Output

### Update or create model

In [ ]:
MODEL_PROVIDER = "openAI"
MODEL_TYPE = "chat"
MODEL_ENABLED = True
MODEL_RESTRICTED = True

models = pg.query(f"SELECT * FROM models WHERE name_alias='{avatar_model_alias}'")

if len(models) == 1:
    model = models.iloc[0]
    res = pg.query(f"UPDATE models SET name='{model}' WHERE id='{model.id}'", commit=True)
elif len(models) == 0:
    res = pg.query(f"INSERT INTO models (name, name_alias, provider, image, enabled, type, restricted) VALUES ('{model}', '{avatar_model_alias}', '{MODEL_PROVIDER}', '{avatar_image}', {MODEL_ENABLED}, '{MODEL_TYPE}', {MODEL_RESTRICTED})", commit=True)
    
models = pg.query(f"SELECT * FROM models WHERE name_alias='{avatar_model_alias}'")
model_id = models.iloc[0].id
model_id

### Link models to users

In [ ]:
WORKSPACE_USERS = [
    "maxime@naas.ai",
    "jeremy@naas.ai",
    "florent@naas.ai"
]

def link_model_to_user(p, model_id, user_id):
    if len(p.query(f"SELECT * FROM restricted_model_user WHERE model_id='{model_id}' AND user_id='{user_id}'")) == 0:
        p.query(f"INSERT INTO restricted_model_user (model_id, user_id) VALUES ('{model_id}', '{user_id}')", commit=True)
        print('✅ User linked')
    else:
        print('✅ User already linked')
        
for user in WORKSPACE_USERS:
    print(user)
    u = pg.query(f"SELECT * FROM users WHERE email='{user}'")
    user_id = u.iloc[0].id
    link_model_to_user(pg, model_id, user_id)